In [ ]:
import sys
# append the path of the parent directory
sys.path.append("..")

# External library imports
import torch
import torch.nn as nn
import numpy as np
import random
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, random_split
import json
import os
import matplotlib.pyplot as plt


# internal library imports

# internal library imports
from dataset import DebrisStateSeriesDataset
from model import ConvLSTM, ConvLSTMComplex, UNetLSTM
from train import TrainerSeries
from util.setting_utils import set_seed


In [ ]:
# Set the random seed for multiple libraries to ensure repeatability

set_seed(42)

In [ ]:
def read_losses(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Extract the subfolder name as the series name
    series_name = os.path.basename(os.path.dirname(file_path))
    return series_name, data['training_losses'], data['validation_losses']

def plot_multiple_losses(files):
    plt.figure(figsize=(12, 6))
    
    for file_path in files:
        series_name, training_losses, validation_losses = read_losses(file_path)
        epochs = range(1, len(training_losses) + 1)
        
        # plt.plot(epochs, training_losses, 'o-', label=f'{series_name} Training', linewidth=2)
        plt.plot(epochs, validation_losses, 'o--', label=f'{series_name} Validation', linewidth=2)

    plt.title('Training and Validation Losses for Multiple Models')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
files = [
    '/home/tom/repos/dyna-landslide-surrogate/checkpoints/2024-04-20_base_model_comparison_l1Loss_batch32_CNN/losses_epoch_30.json',
    '/home/tom/repos/dyna-landslide-surrogate/checkpoints/2024-04-20_base_model_comparison_l1Loss_batch32_SmallUNet/losses_epoch_30.json',
    '/home/tom/repos/dyna-landslide-surrogate/checkpoints/2024-04-20_base_model_comparison_l1Loss_batch32_MedUNet/losses_epoch_30.json',
    '/home/tom/repos/dyna-landslide-surrogate/checkpoints/2024-04-21_base_model_comparison_l1Loss_batch32_LargeUNet/losses_epoch_30.json'
]

plot_multiple_losses(files)

In [ ]:
# Parameters

root_dir = r'/home/tom/repos/dyna-landslide-surrogate/data_small'
checkpoint_dir = r'/home/tom/repos/dyna-landslide-surrogate/checkpoints'
batch_size = 64
split_proportions = (0.7, 0.15, 0.15)
epochs = 5
seq_length = 5  # Specify the sequence length for the ConvLSTMUNet
array_size = 256

in_channels = 3  # Number of input channels (e.g., terrain, velocity, thickness)
out_channels = 2  # Number of output channels (e.g., next velocity, next thickness)
hidden_dim = 64  # Hidden dimension of the ConvLSTM
kernel_size = (3, 3)  # Kernel size of the ConvLSTM

In [ ]:
# Data set up

# Initialize dataset with scaling
dataset = DebrisStateSeriesDataset(root_dir=root_dir, array_size=array_size, apply_scaling=True, sequence_length=seq_length)

# Split dataset into train, validation, and test sets and create dataloaders
train_loader, val_loader, test_loader = dataset.create_dataloaders(split_proportions, batch_size, random_state=42)

# Dataset stats
print(f"Total dataset size: {len(dataset)}")
print(f"Train size: {len(train_loader.dataset)}, Validation size: {len(val_loader.dataset)}, Test size: {len(test_loader.dataset)}")

print(dataset.scaling_factors)

In [ ]:
# Create an instance of each architecture
cnn = CNN()
small_unet = UNet(in_channels=3, out_channels=2, features=[64, 128, 256])
med_unet = UNet(in_channels=3, out_channels=2, features=[64, 128, 256, 512])
large_unet = UNet(in_channels=3, out_channels=2, features=[64, 128, 256, 512, 1024])

# Calculate the number of parameters for each architecture
cnn_params = sum(p.numel() for p in cnn.parameters())
small_unet_params = sum(p.numel() for p in small_unet.parameters())
med_unet_params = sum(p.numel() for p in med_unet.parameters())
large_unet_params = sum(p.numel() for p in large_unet.parameters())

# Print the number of parameters for each architecture
print("ComplexCNN parameters:", cnn_params)
print("SimpleUNet parameters:", small_unet_params)
print("LargeUNet parameters:", med_unet_params)
print("UNet parameters:", large_unet_params)

In [ ]:
# Set up CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unet_lstm = UNetLSTM().to(device)

# Check if multiple GPUs are available and wrap the model using nn.DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    # This will wrap the model for use with multiple GPUs
    unet_lstm = nn.DataParallel(unet_lstm)

# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
# criterion = CustomDebrisLoss(loss_fn_zero=nn.SmoothL1Loss(), loss_fn_debris=nn.SmoothL1Loss(), debris_weight=0.66)
optimizer = Adam(unet_lstm.parameters(), lr=1e-3)

unet_lstm_trainer = TrainerSeries(unet_lstm, optimizer, criterion, device, model_name="cnn", checkpoint_dir=checkpoint_dir)

unet_lstm_trainer.load_checkpoint("/home/tom/repos/dyna-landslide-surrogate/checkpoints/unet_lstm_test_int5_seq5_fulldataprune/model_epoch_5.pth", train_loader)

In [ ]:
unet_lstm_trainer.test(test_loader)

In [ ]:
# unet_lstm_trainer.plot_predictions(test_loader, num_predictions=5)

In [ ]:
def plot_real_vs_inferred(real_states, inferred_states, n=5):
    num_timesteps = len(real_states)
    timesteps_to_plot = range(1, num_timesteps + 1, n)  # Start from 1 and go up to num_timesteps + 1

    for timestep in timesteps_to_plot:
        real_state = real_states[timestep]
        inferred_state = inferred_states[timestep]

        # Extract velocity and thickness from real and inferred states
        real_velocity = real_state[1]
        real_thickness = real_state[0]
        inferred_velocity = inferred_state[1]
        inferred_thickness = inferred_state[0]

        # Calculate differences
        velocity_diff = np.abs(real_velocity - inferred_velocity)
        thickness_diff = np.abs(real_thickness - inferred_thickness)

        plt.figure(figsize=(20, 8))

        # Row 1: Real Velocity, Inferred Velocity, Velocity Difference
        plt.subplot(2, 3, 1)
        plt.imshow(real_velocity, cmap='jet')
        plt.title(f'Real Velocity (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        plt.subplot(2, 3, 2)
        plt.imshow(inferred_velocity, cmap='jet')
        plt.title(f'Inferred Velocity (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        plt.subplot(2, 3, 3)
        plt.imshow(velocity_diff, cmap='jet')
        plt.title(f'Velocity Difference (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        # Row 2: Real Thickness, Inferred Thickness, Thickness Difference
        plt.subplot(2, 3, 4)
        plt.imshow(real_thickness, cmap='jet')
        plt.title(f'Real Thickness (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        plt.subplot(2, 3, 5)
        plt.imshow(inferred_thickness, cmap='jet')
        plt.title(f'Inferred Thickness (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        plt.subplot(2, 3, 6)
        plt.imshow(thickness_diff, cmap='jet')
        plt.title(f'Thickness Difference (Timestep {timestep})')
        plt.axis('off')
        plt.colorbar()

        plt.tight_layout()
        plt.show()

    # Stack all inferred arrays and all actual arrays
    stacked_inferred_velocity = np.stack([state[1] for state in inferred_states.values()], axis=0)
    stacked_inferred_thickness = np.stack([state[0] for state in inferred_states.values()], axis=0)
    stacked_real_velocity = np.stack([state[1] for state in real_states.values()], axis=0)
    stacked_real_thickness = np.stack([state[0] for state in real_states.values()], axis=0)

    # Plot stacked arrays side by side
    plt.figure(figsize=(20, 8))

    plt.subplot(2, 3, 1)
    plt.imshow(stacked_real_velocity.max(axis=0), cmap='jet')
    plt.title('Stacked Real Velocity')
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 3, 2)
    plt.imshow(stacked_inferred_velocity.max(axis=0), cmap='jet')
    plt.title('Stacked Inferred Velocity')
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 3, 3)
    plt.imshow(np.abs(stacked_real_velocity.max(axis=0) - stacked_inferred_velocity.max(axis=0)), cmap='jet')
    plt.title('Stacked Velocity Difference')
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 3, 4)
    plt.imshow(stacked_real_thickness.max(axis=0), cmap='jet')
    plt.title('Stacked Real Thickness')
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 3, 5)
    plt.imshow(stacked_inferred_thickness.max(axis=0), cmap='jet')
    plt.title('Stacked Inferred Thickness')
    plt.axis('off')
    plt.colorbar()

    plt.subplot(2, 3, 6)
    plt.imshow(np.abs(stacked_real_thickness.max(axis=0) - stacked_inferred_thickness.max(axis=0)), cmap='jet')
    plt.title('Stacked Thickness Difference')
    plt.axis('off')
    plt.colorbar()

    plt.tight_layout()
    plt.show()


def create_real_states_dict(root_dir, model_number, start_state, num_timesteps, array_size):
    real_states = {}
    
    model_dir = os.path.join(root_dir, str(model_number))
    velocity_dir = os.path.join(model_dir, f'04_FinalProcessedData_{array_size}', 'velocity')
    thickness_dir = os.path.join(model_dir, f'04_FinalProcessedData_{array_size}', 'thickness')
    
    for t in range(num_timesteps):
        state_number = start_state + t
        
        velocity_file = os.path.join(velocity_dir, f'{model_number}_velocity_{state_number}.npy')
        thickness_file = os.path.join(thickness_dir, f'{model_number}_thickness_{state_number}.npy')
        
        velocity = np.load(velocity_file)
        thickness = np.load(thickness_file)
        
        real_states[t + 1] = np.stack((thickness, velocity), axis=0)  # Change the key to t + 1
    
    return real_states

In [ ]:
model_id = '00194'


# Create the initial input
initial_input = unet_lstm_trainer.create_inference_input(root_dir, model_id, 10, 256)

# Perform inference
num_timesteps = 30
inferred_states = unet_lstm_trainer.infer(initial_input, num_timesteps)

# Create the real states dictionary
real_states = create_real_states_dict(root_dir, model_id, num_timesteps+1, num_timesteps, 256)

# Plot real vs inferred states
plot_real_vs_inferred(real_states, inferred_states, n=5)

In [ ]:
type(initial_input)

In [ ]:
initial_input.shape

In [ ]:
inferred_states

In [ ]:
inferred_states[1].shape

In [ ]:
for key, array in inferred_states.items():
    min_value = np.min(array)
    max_value = np.max(array)
    print(f"Min and max values for {key}: ({min_value}, {max_value})")

In [ ]:


# Create the initial input
initial_input = med_unet_trainer.create_inference_input(root_dir, '00204', 2, 256)

# Perform inference
num_timesteps = 30
inferred_states = med_unet_trainer.infer(initial_input, num_timesteps)

# Create the real states dictionary
real_states = create_real_states_dict(root_dir, '00204', 2 + 1, num_timesteps, 256)

# Plot real vs inferred states
plot_real_vs_inferred(real_states, inferred_states, n=5)